## 论文简介  

全卷积网络(Fully Convolutional Network)将 CNN 应用到了图像语义分割领域。图像语义分割，就是对一张图片上的所有像素点进行分类。

以往的 CNN 都是对整张图片进行分类,不能识别图片中特定部分的物体，而全卷积网络是对一张图片中的每个像素进行分类，以此达到对图片特定部分进行分类的效果。  

本文提出了全卷积网络(FCN)的概念，针对语义分割训练一个端到端，点对点的网络，达到了state-of-art。这是第一次训练端到端的 FCN，用于像素级的预测；也是第一次用监督预训练的方法训练 FCN。  

### 网络结构  

#### 1 卷积化  

分类所使用的网络通常会在最后连接全连接层，它会将原来二维的矩阵(图片)压缩成一维的，从而丢失了空间信息，最后训练输出一个标量，作为最终的分类结果。  

而图像语义分割的输出则需要是个分割图，且不论尺寸大小，但是至少是二维的。卷积化就是将网络的全连接层换成卷积层的操作。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/FCN/2.jpg?raw=true)

假设卷积神经网络的输入是224x224x3的图像，一系列的卷积层和下采样层将图像数据变为尺寸为 7x7x512 的数据体。AlexNet 使用了两个尺寸为4096的全连接层，最后一个有1000个神经元的全连接层用于计算分类评分。  

下面将这3个全连接层转化为卷积层：  

第一个全连接层:原来的参数是7x7x512x4096，这里用4096个7x7的filter替换原来的全连接层，这样输出就是[1x1x4096]了； 
第二个全连接层:4096个1x1的filter，这样输出为[1x1x4096]； 
第三个全连接层:1000个1x1的filter，最终输出为[1x1x1000]。  

#### 2 上采样  

**上采样就是将随后得到的下采样之后的特征图映射回原图大小的分割图的过程。**  

在一般的CNN结构中,如 AlexNet,VGGNet 均是使用池化层来缩小输出图片的 size，例如 VGG16,五次池化后图片被缩小了32倍;而在ResNet中，某些卷积层也参与到缩小图片size的过程。我们需要得到的是一个与原图像大小相同的分割图，因此需要对最后一层进行上采样，一般通过反卷积(Deconvolution)的操作进行上采样(或称为转置卷积(conv_transpose))。  

**虽然转置卷积层和卷积层一样，也是可以学习卷积层的参数的，但实际实验中，作者发现，让转置卷积层可学习，并没有带来效果的提升，所以实验中的转置卷积层的 lr 全部被置零了。**  

作者在github上面解释，初始化转置卷积层的方法是双线性插值。论文中最开始反卷积层是可以学习的，但是后来实验发现把学习率置为零对效果没有影响，还可以加速训练，所以就置零了。

**注意:**  
为了得到和输入图像尺寸完全相同的特征图,FCN 中还使用了 crop 操作来辅助转置卷积操作，因为转置卷积操作并不是将特征图恰好放大整数倍。  

#### 3 跳跃结构  

其实直接使用前两种结构就已经可以得到结果了，但是直接将全卷积后的结果上采样后得到的结果通常是很粗糙的。所以这一结构主要是用来优化最终结果的，思路就是将不同池化层的结果进行上采样，然后结合这些结果来优化输出，具体结构如下:  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/FCN/6.jpg?raw=true)

如上图所示，对原图像进行卷积 conv1、pool1 后原图像缩小为 1/2；之后对图像进行第二次 conv2、pool2 后图像缩小为 1/4；接着继续对图像进行第三次卷积操作 conv3、pool3 缩小为原图像的 1/8，此时保留 pool3 的 featureMap；接着继续对图像进行第四次卷积操作 conv4、pool4，缩小为原图像的1/16，保留 pool4 的 featureMap；最后对图像进行第五次卷积操作 conv5、pool5，缩小为原图像的 1/32，然后把原来CNN操作中的全连接变成卷积操作conv6、conv7，图像的 featureMap 数量改变但是图像大小依然为原图的 1/32，此时图像不再叫 featureMap 而是叫 heatMap。
 
现在我们有 1/32 尺寸的 heatMap，1/16 尺寸的 featureMap 和1/8尺寸的 featureMap，1/32 尺寸的 heatMap 进行 upsampling 操作之后，因为这样的操作还原的图片仅仅是 conv5 中的卷积核中的特征，限于精度问题不能够很好地还原图像当中的特征，因此在这里向前迭代。把 conv4 中的卷积核对上一次 upsampling 之后的图进行反卷积补充细节（相当于一个插值过程），最后把 conv3 中的卷积核对刚才 upsampling 之后的图像进行再次反卷积补充细节，最后就完成了整个图像的还原。  

不同的结构产生的结果对比如下:  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/FCN/4.jpg?raw=true)

## 4 网络结构如图所示  



## Abstract  

卷积网络是强大的视觉模型，可以产生不同层次的特征。我们表明，通过端到端的训练卷积网络本身，在语义分割任务上可以超过目前最先进的方法。关键洞察是建立“全卷积”网络，以任意大小的输入和有效的推理产生相应大小的输出。我们定义并详细说明全卷积网络，解释它们在空间密集预测任务中的应用，并且绘制与先前模型的连接。我们将目前存在的分类网络（AlexNet，VGG 和 GoogLeNet）应用于全卷积网络，并通过微调将它们的学习表示转移到分割任务。然后，我们设计一种新颖的架构，将来自深层粗略层次的语义信息与浅层精细层次的外观信息相结合，以产生精确细致的分割。我们的全卷积网络实现了PASCAL VOC（相对于2012年的平均IU提高20％至平均IU 62.2％），NYUDv2 和 SIFT Flow 的最先进的分割，而对于典型的图像，推断仅需不到五分之一秒。

## 1. Introduction  

Convnets 不仅改进了整体图像分类的效果，而且在结构化输出的定位任务上也取得了进展。这些包括边界框对象检测，关键点预测和定位方面的进展。  

从粗到细推理的下一步是对每个像素进行预测。先前已经有文章将卷积网络用于语义分割，每个像素点用包围其的对象或区域类别进行标注，但是这种方法不管是在速度上还是精度上都有很大的缺陷。  

我们展示了一个全卷积网络（FCN），在语义分割上进行端到端的像素到像素的训练，超越了现有技术水平。据我们所知，这是第一个针对像素点预测端到端训练训练 FCN的工作 （1）像素点预测（2）有监督预训练。全卷积在现有的网络基础上从任意尺寸的输入预测密集输出。学习和推理能在全图通过密集的前馈计算和反向传播一次执行。网内上采样层能在像素级别预测和通过下采样池化学习。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/FCN/1.jpg?raw=true)

这种方法既快速又有效，并且不需要太多的复杂性。分批训练是常见的方法，但缺乏全卷积训练的有效性。我们的方法不利用复杂的预处理或后处理，包括超像素，区域提议或随机细化或局部分类。我们的模型通过将分类网络重新调整为全卷积网络并且从这些网络已经学习到的参数中微调全卷积网络，从而将分类认为转化为密集预测任务。相比之下，以前的方法用了没有经过有监督预训练的小网络。  

语义分割面临着语义和位置之间固有问题：全局信息解决了是什么的问题，局部信息解决了在哪里的问题。深层特征通过非线性的局部到全局金字塔编码了位置和语义信息。我们定义了一个新颖的“skip”体系结构，在4.2节（见图3）中结合了深、粗层次的语义信息和浅、精细的表征信息。  

在下一节中，我们将回顾有关深度分类网络，FCNs 和近期使用小网格进行语义分割的方法。以下部分解释了 FCN 设计和密集预测de 折衷方案，将我们的架构与网内上采样和多层组合相结合，并描述了我们的实验框架。最后，展示我们在 PASCAL VOC 2011-2，NYUDv2 和 SIFT Flow 的最新成果  

## 2. Related work  

我们的方法利用了深度网络在图像分类和迁移学习方面取得的最新成果。迁移先在各种视觉识别任务上得到应用，然后在检测、以及用混合区域提议分类器模型实现的实例和语义分割中。我们现在重新设计和微调分类网络来指导语义分割的密集预测任务。我们绘制了 FCN 的空间，并在这个框架中放置了历史和近期的先前模型。  

#### Fully convolutional networks  

就我们所知，第一次将卷积网扩展到任意尺寸的输入的是Matan等人，它扩展了经典的 LeNet 来识别数字字符串。Wolf 和 Platt 将卷积网输出扩展为二维图，用来定位邮政地址块的四个角。Ning等人将全卷积网络用于秀丽线虫组织的粗糙的、多分类分割任务。  

全卷积计算也被用在现在的一些多层次的网络结构中。 Sermanet 等人的滑动窗口检测，Pinheiro 和 Collobert 的语义分割，以及 Eigen 等人的图像恢复，都是基于全卷积网络的。全卷积训练很少，但是被 Tompson 等人用来学习一种非常有效的端到端的局部检测和姿态估计的空间模型，尽管他们没有解释或者分析这种方法。  

另外，He 等人放弃分类网络的非卷积部分来做特征提取。他们结合 proposals 和空间金字塔 pooling 来产生一个局部的、固定长度的特征用于分类。尽管快速且有效，但是这种混合模型不能进行端到端的学习。  

#### Dense prediction with convnets  

近期的一些工作已经将卷积网应用于密集预测任务，包括 Ning 等人的语义分割 和 Ciresan 等人对电子显微镜的边界预测。以及Ganin 和 Lempitsky 通过混合卷积网和最邻近模型处理自然场景图像;还有 Eigen 等人的图像修复和深度估计。这些方法的相同点包括如下：
* 限制容量和接收域的小模型
* patchwise 训练
* 超像素投影的预处理，随机场正则化、滤波或局部分类
* 像 OverFeat 介绍的输入移位和输出交错的密集输出
* 多尺度金字塔预处理
* 饱和的 tanh 激活函数
* 集成

我们的方法没有这些机制。但我们研究了 patchwise 训练和从 FCNs 的角度出发的“shift-and-stitch”密集输出。也讨论了网内采样。  

与现有方法不同，我们采用图像分类作为有监督的预训练来适应和扩展深度分类体系结构，并且全卷积地进行微调以从整个图像输入和整个图像 ground thruths 简单而有效地学习。  

Hariharan 等人和 Gupta 等同样用深层分类网络进行语义分割，但在基于混合 proposal-classifier 模型。这些方法通过对边界框和区域提议进行抽样来来微调 R-CNN 系统，从而进行检测，语义分割和实例分割等任务。这两种方法都不是端到端学习的。  

他们分别在 PASCAL VOC 分割和 NYUDv2 分割方面取得了最新的成果，所以我们直接比较了我们的独立端到端 FCN 和他们的语义分割结果。  

## 3. Fully convolutional networks  

convnet 中的每一层数据都是大小为 h×w×d 的三维数组，其中 h 和 w 是空间维度，d 是特征或通道维度。第一层是图像，像素大小为 h×w，d 个颜色通道。更高层的位置对应于它们所路径连接的图像中的位置，这些位置被称为接受域。  

Convnets 是以平移不变形作为基础的。它们的基本组件（convolution, pooling, and activation functions）在局部输入区域上运行，并且仅依赖于相对空间坐标。在特定层的位置 $(i,j)$ 的数据向量为 $X_{ij}$x，下一层中为 $Y_{ij}$。    

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/FCN/2.jpg?raw=true)

一个由 FCN 组成的实值损失函数定义一个任务。如果损失函数是最终层的空间维数的总和 $l(x,\theta) =  \sum_{ij} l'(X_{ij};\theta)$，则其梯度将是每个空间分量的梯度之和。因此，在全部图像上基于 $l$ 计算的随机梯度下降将与在 $l'$ 上的随机梯度下降相同，将最后一层的所有接收域作为minibatch（分批处理）。  

当这些感受域重叠较多时，前反馈计算和反向传播计算整图的叠层都比独立的patch-by-patch有效的多。  

接下来，我们将解释如何将分类网转换为能输出粗 coarse output maps 的全卷积网络。对于像素级预测，我们需要将这些粗略的输出连接回像素。第 3.2节描述了 OverFeat 为此目的而引入的一个技巧。我们通过将它解释为一个等价网络修正而获得了关于这个技巧的一些领悟。作为一种有效的替代方法，我们在 3.3 节介绍了用于上采样的去卷积层。在 3.4 节中，我们考虑采用 patchwise 抽样的方法进行训练，并在 4.3 节给出证据，证明我们的整个图像训练速度更快，效果也更好。  

### 3.1. Adapting classifiers for dense prediction  

典型的识别网络，包括LeNet，AlexNet 及其更深的继承者，表面上采用固定大小的输入并产生非空间输出。这些网络的全连接层具有固定的尺寸并丢弃空间坐标。然而，这些全连接层也可以被视为与覆盖整个输入区域的卷积。这样做把它们映射到全卷积网络中，输入任意大小和输出分类图。这种变化如图2所示。  

此外，当作为结果的图在特殊的输入patches上等同于原始网络的估计，在这些小片的重叠区域上的计算是高度分摊的。例如，虽然 AlexNet 需要1.2 ms（在典型的GPU上）产生 227×227 图像的分类分数，但全卷积版本需要 22 ms 来从 500×500 图像产生 10×10 个输出的网格，比原来的方法快 5 倍以上。  

这些卷积模型的空间输出图使得它们成为诸如语义分割之类的密集问题的自然选择。在每个输出单元都具有 ground truth  的情况下，前向和后向过程都是直接的，都利用了卷积的固有计算效率。  

AlexNet 示例的相应后向时间是单个图像为 2.4 ms，全卷积 10 x 10 输出映射为 37 ms，类似于前向的加速。这种密集的反向传播在图 1 中示出。  

尽管我们将分类网络重新解释为全卷积产生任意大小的输入的输出映射，但输出维度通常通过下采样来减少。分类网络下采样保持小的 filter 和 合理的计算需求。这使全卷积式网络的输出结果变得粗糙，通过输入尺寸因为一个和输出单元的接收域的像素步长等同的因素来降低它。  

### 3.2. Shift-and-stitch is filter rarefaction  

输入移位和输出交错是一个由OverFeat引入的技巧，可以从没有插值的粗输出中产生密集的预测。如果输出采样的因子为 f，则输入向左（右侧和顶部填充）移动 x 个像素，向下移动 y 个像素，$(x,y) \in {0,...,f-1} * {0,...,f-1}$。这 $f^2$ 个输入每个都通过 convnet 运行，并且输出是交错的，以便预测对应于其接受场中心的像素。    

虽然我们已经做了 shift-and-stitch 的初步实验，但是我们并没有在模型中使用它。如下一节所述，我们发现通过上采样进行学习是更加有效和高效的，特别是当与后面描述的 skip 层融合相结合时。  

### 3.3. 上采样是向后向卷积  

将粗输出连接到密集像素的另一种方法是插值。例如，简单的双线性插值通过线性映射来计算来自最近四个输入的每个输出$y_{ij}$，线性映射仅依赖于输入和输出单元的相对位置。  

从某种意义上讲，f 因子的上采样是 1/f 输入步长的卷积。只要 f 是整数，上采样的一个自然的方法就是向后卷积（有时称为解卷积）伴随输出步长为f。这样的操作实现起来是微不足道的，因为它简单地颠倒了卷积的前向和后向过程。所以上采样在网内通过计算像素级别的损失的反向传播用于端到端的学习。  

注意，这样的层中的去卷积滤波器不需要是固定的（例如，对于双线性上采样），但是可以被学习。 一堆去卷积层和激活函数甚至可以学习非线性上采样。  

在我们的实验中，我们发现网络上采样对于学习密集预测是快速有效的。 我们最好的分割体系结构使用这些图层来学习在4.2节中进行精细预测的上采样。  

### 3.4. Patchwise 训练是一种损失采样  

在随机优化中，梯度计算是由训练分布驱动的。patchwise 训练和全卷积训练能被用来产生任意分布，尽管他们相对的计算效率依赖于重叠域和 minibatch的大小。在每一个由所有的单元接受域组成的批次在图像的损失之下（或图像的集合）整张图像的全卷积训练等同于 patchwise 训练。虽然这比均匀采样更有效，但它减少了可能的批次数量。然而在一张图片中随机选择 patches 可能更容易被重新找到。限制基于它的空间位置随机取样子集产生的损失（或者可以说应用输入和输出之间的DropConnect mask [39] ）排除来自梯度计算的patches。  

如果保存的 patches 仍然有明显的重叠，全卷积计算仍然会加快训练速度。  

抽样训练中的采样可以纠正类别失衡，并减轻密集斑块的空间相关性。在全卷积训练中，类别平衡也可以通过权重损失来实现，损失采样可以用来解决空间相关性。  

在 4.3 节中我们研究了有采样的训练，没有发现对于 dense prediction有更快或是更好的收敛效果。全图训练是有效且高效的。  

## 4. Segmentation Architecture  

我们将 ILSVRC 分类器投射到 FCN 中，并将它们用于网络上采样和像素损失的密集预测。我们通过微调来训练分割。接下来，我们构建了一个新颖的 skip 体系结构，将粗糙的，语义的和局部的外观信息结合起来进行预测。  

对于这项调查，我们在 PASCAL VOC 2011 分割挑战上进行训练和验证。我们用每像素多项式逻辑损失进行训练，并使用平均像素相交的标准度量来验证，包括背景在内的所有类别均值。训练忽略在 ground truth 中被掩盖掉（像模棱两可或困难）的像素。  

### 4.1. From classifier to dense FCN  

我们从卷积化成熟的分类体系结构开始，如第3节所述。我们考虑赢得 ILSVRC12 的 AlexNet 体系结构，以及在 ILSVRC14 中做得非常好的 VGG 网络和GoogLeNet。我们选择了 VGG 的 16 层 net，发现它相当于这个任务的 19 层网络。对于 GoogLeNet，我们只使用最后的 loss 层，并通过丢弃最终的平均池层来提高性能。通过丢弃最终的分类层来斩断每个网络，并将所有全连接的层转换为卷积。我们添加 21 通道的 1 × 1 filters 来预测每个粗略输出位置的每个 PASCAL 类别（包括背景）的分数，然后是一个去卷积层，将粗略输出双线性上采样到如在 3.3节 所述的像素密集输出。表1 比较了初步验证结果和每个网络的基本特征。我们汇报了固定学习率（至少175个 epochs）收敛后取得的最好结果。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/FCN/3.jpg?raw=true)

从分类到分割的微调给每个网络提供了合理的预测。即使是最糟糕的模式也达到了75％的最先进表现。分割的的 VGG 网络（FCN-VGG16）在 val 上的平均IU 值为 56.0，而在测试中为 52.6。  

尽管类似的分类准确性，我们的 GoogLeNet 的实现不符合这种分割结果。  

### 4.2. Combining what and where  

我们定义了一个新的全卷积网用于结合了特征层级的分割并提高了输出的空间精度。见图3  

虽然全卷积分类器可以如 4.1 所示进行细化分割，甚至在标准度量上得分也很高，但是它们的输出却不尽人意（见图4）。最终预测层的 32 像素步长限制了上采样输出的细节范围。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/FCN/4.jpg?raw=true)

我们提出增加结合了最后预测层和有更细小步长的更低层的跨层信息来解决这个问题。将一个线划拓扑结构转变成DAG(有向无环图)，其边缘从较低层跳到较高层（图3）。因为它们只能获取更少的像素点，更精细的尺寸预测应该需要更少的层，所以从更浅的网中将它们输出是有道理的。结合了精细层和粗糙层让模型能做出遵从全局结构的局部预测。与Koenderick 和an Doorn [21]的jet类似，我们把这种非线性特征层称之为deep jet。  

首先将输出步长分为一半，通过一个16像素步长层预测。我们在 pool4 的顶部增加了一个 1×1 的卷积层来产生附加的类别预测。我们将输出和预测融合在 conv7（fc7的卷积化）的顶部以步长32计算，通过增加一个2×的上采样层和预测求和（见图3）。我们初始化这个2×上采样到双线性插值，但是允许参数能被学习，如3.3节所描述、最后，步长为16的预测被上采样回图像，我们把这种网结构称为FCN-16s。FCN-16s用来学习端到端，能被最后的参数初始化。这种新的、在pool4上生效的参数是初始化为0 的，所以这种网结构是以未变性的预测开始的。这种学习速率是以100倍的下降的。  

学习这个 skip net 将验证集上的性能提高了3.0，即平均IU为62.4。图4显示了输出精细结构的改进。我们将这种融合学习和仅仅从pool4层上学习进行比较，结果表现糟糕，而且仅仅降低了学习速率而没有增加跨层，导致了没有提高输出质量的没有显著提高表现。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/FCN/5.jpg?raw=true)
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/FCN/6.jpg?raw=true)

#### Refinement by other means  

减少 pooling 层的 stride 是获得更好预测的最直接的方法。但是，对于基于 VGG16 的网络来说，这样做是有问题的。将 pool5 图层设置为 stride=1 要求我们的卷积化 fc6 具有 14×14 的大小，以保持其 receptive field 大小。除了计算成本之外，我们很难学习如此大的 filter。我们试图用较小的滤波器重新构建 pool5 以上的层，但是在实现可比较的性能方面没有成功; 一个可能的解释是从上层的 ImageNet 训练权重的初始化是重要的。  

获得更好预测的另一种方法是使用3.2节中描述的 shift-and-stitch 技巧。在有限的实验中，我们发现这种方法的改进成本比层融合要差。  

### 4.3. Experimental framework  

#### Optimization  

我们用有动量的 SGD 训练。对于FCN-AlexNet，FCN-VGG16 和 FCN-GoogLeNet，分别使用 20 个图像的 minibatch 大小和$10^{-3}$,$10^{-4}$和$5^{-5}$ 的固定学习速率，通过行搜索选择。尽管我们发现训练对这些参数不敏感（但对学习速率敏感），我们使用动量0.9，weight decay 为 $5^{-4}$ 或 $2^{-4}$，并将偏差的学习速率加倍。我们对类评分卷积层进行零初始化，发现随机初始化以产生更好的性能和更快的收敛性。包括在原分类网中使用的Dropout。  

#### Fine-tuning  

我们通过全网反向传播来微调所有层。单独对输出分类器进行微调只会产生完全微调性能的70％，如表2所示。考虑到学习基本分类网络所需的时间，从零开始进行训练是不可行的。（请注意，VGG网络是分阶段训练的，而我们是从完整的16层版本初始化的）对于coarse FCN-32 版本，在单个GPU上进行微调需要三天。  

#### Patch Sampling  

如第3.4节所述，我们的全图像训练有效地将每个图像分成一个规则的大网格，重叠的 patches。相比之下，先前的工作在整个数据集上随机采样，可能导致更高的方差patches，可能会加速收敛。我们通过以前面描述的方式对损失进行空间抽样来研究这种折衷，做出一个独立的选择，以一定的概率 1-p 忽略每个最后的层单元。为了避免改变有效的批次尺寸，我们同时将每批次的图像数量增加 1/p。注意的是因为卷积的效率，在足够大的 p 值下，这种拒绝采样的形式仍然比 patchwise 训练快。图5显示了这种采样形式对收敛的影响。我们发现采样在收敛速率上没有很显著的效果相对于全图式训练，但是由于每个每个批次都需要大量的图像，很明显的需要花费更多的时间。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/FCN/7.jpg?raw=true)

#### Class Balancing  

全卷积训练可以通过对损失进行加权或抽样来平衡数据。虽然我们的标签是轻微不平衡的（大约3/4是背景），但我们发现类别平衡是不必要的。  

#### Dense Prediction  

通过网络内的解卷积层将分数上采样到输入维度。最终层去卷积滤波器被固定为双线性插值，而中间上采样层被初始化为双线性上采样，然后被学习。  

#### More Training Data  

我们用于表1的 PASCAL VOC 2011 分割挑战训练集标注了 1112 幅图像。 Hariharan等人已经收集了更大的一套 8498 PASCAL 训练图像的标签，用于训练先前的最先进的系统 SDS。该训练数据将 FCNVGG16 验证得分提高了 3.4 分，达到59.4.  

#### Implementation  

所有版本都经过 Caffe 训练和测试，使用单 个NVIDIA Tesla K40c。

## 5. Results  

我们在语义分割和场景分析任务中测试我们的 FCN，研究了 PASCAL VOC，NYUDv2 等数据。尽管这些任务在以前主要是用在物体和区域上，我们都将它们视为像素预测。  

#### Metrics  

我们从常见语义分割和场景分析评估中报告四个度量，这些评估是像素精度和区域交集（IU）上的变化。令$n_{ij}$为第 i 类预测属于第 j 类的像素的数目，其中有 $n_{cl}$ 个不同的类别，令 $t_i = \sum_{j}n_{ij}$ 为第 i 类像素的总数。 我们计算：  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/FCN/8.jpg?raw=true) 

#### PASCAL VOC  

表3给出了我们的 FCN-8 在 PASCAL VOC 2011 和 2012 测试集上的性能，并将其与以前最先进的 SDS 和著名的R-CNN。我们在平均 IU上取得了最好的结果，相对提升了20％。推断时间减少了114倍（仅限于细分，忽略建议和细化）或者286倍（总体）。  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/FCN/9.jpg?raw=true)
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/FCN/10.jpg?raw=true)

## 6. Conclusion  

全卷积网络是一个丰富的模型类别，是现代化分类网络中一个特殊的例子。认识到这一点，将这些分类网络扩展到分割，并通过多分辨率层组合改进体系结构，大大提高了现有技术水平，同时简化和加快了学习和推理。

## 参考  

1 [Fully Convolutional Networks for Semantic Segmentation](https://arxiv.org/abs/1411.4038)  
2 [FCN学习:Semantic Segmentation](https://zhuanlan.zhihu.com/p/22976342)  